# Full Atmospheric Correction Workflow

This notebook demonstrates the complete atmospheric correction workflow using the `AtmosphericCorrection` class. We'll walk through:

1. Setting up input data (TOA radiances, geometry, ancillary data)
2. Running the full atmospheric correction
3. Understanding the outputs (Rrs, nLw, chlorophyll)
4. Quality flags and diagnostics

The atmospheric correction transforms **top-of-atmosphere radiance (Lt)** into **remote sensing reflectance (Rrs)** and **normalized water-leaving radiance (nLw)**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from correct_atmosphere import constants
from correct_atmosphere.correction import (
    AtmosphericCorrection,
    GeometryAngles,
    AncillaryData,
    CorrectionResult,
    CorrectionFlags
)

---
## 1. The Atmospheric Correction Equation

The fundamental equation for ocean color atmospheric correction is:

$$L_t = L_r + L_a + t L_g + t L_{wc} + t L_w$$

where:
- $L_t$ = Total radiance at TOA (what the sensor measures)
- $L_r$ = Rayleigh path radiance
- $L_a$ = Aerosol path radiance
- $L_g$ = Sun glint radiance
- $L_{wc}$ = Whitecap radiance
- $L_w$ = Water-leaving radiance (the signal we want!)
- $t$ = Atmospheric transmittance

The goal is to solve for $L_w$ by accurately estimating and removing all other terms.

---
## 2. Initialize the Atmospheric Correction Processor

Create an `AtmosphericCorrection` instance for your sensor. Available sensors:
- `seawifs`
- `modis_aqua`
- `modis_terra`
- `viirs_npp`
- `viirs_noaa20`

In [ ]:
# Create processor for MODIS-Aqua
sensor = 'modis_aqua'
ac = AtmosphericCorrection(sensor)

print(f"Atmospheric Correction Processor")
print(f"================================")
print(f"Sensor: {ac.sensor}")
print(f"\nSpectral bands (nm):")
for i, wavelength in enumerate(ac.wavelengths):
    print(f"  Band {i+1}: {wavelength} nm")

---
## 3. Prepare Input Data

The correction requires three types of input:
1. **TOA radiances** - Dictionary of radiance arrays keyed by band name
2. **Geometry** - Solar and viewing angles
3. **Ancillary data** - Environmental parameters

### 3.1 Create Synthetic TOA Radiances

For this demonstration, we'll create realistic synthetic TOA radiances for different water types.

In [ ]:
def create_synthetic_toa_radiance(water_type='clear_ocean', geometry=None, ancillary=None):
    """
    Create synthetic TOA radiances for demonstration.
    
    Parameters
    ----------
    water_type : str
        'clear_ocean', 'productive', or 'turbid'
    
    Returns
    -------
    ndarray : TOA radiances in mW/cm^2/um/sr, shape (n_bands,)
    """
    # Wavelengths matching MODIS-Aqua bands
    wavelengths = np.array([412, 443, 490, 510, 555, 670, 765, 865])
    
    # Typical Rayleigh path radiance (decreases with wavelength)
    Lr = np.array([8.0, 6.0, 4.0, 3.2, 2.0, 0.8, 0.4, 0.2])
    
    # Typical aerosol path radiance
    La = np.array([1.0, 0.9, 0.8, 0.75, 0.65, 0.5, 0.4, 0.35])
    
    # Water-leaving radiance depends on water type
    if water_type == 'clear_ocean':
        # Deep blue water - high blue, low green/red
        Lw = np.array([0.8, 1.0, 0.7, 0.4, 0.2, 0.02, 0.0, 0.0])
    elif water_type == 'productive':
        # Phytoplankton-rich - lower blue, higher green
        Lw = np.array([0.3, 0.4, 0.6, 0.7, 0.6, 0.1, 0.01, 0.0])
    elif water_type == 'turbid':
        # Sediment-laden - high across visible, some NIR
        Lw = np.array([0.5, 0.6, 0.8, 1.0, 1.5, 0.8, 0.2, 0.05])
    else:
        raise ValueError(f"Unknown water type: {water_type}")
    
    # Atmospheric transmittance (simplified)
    t = np.array([0.75, 0.78, 0.82, 0.84, 0.87, 0.92, 0.94, 0.95])
    
    # Total TOA radiance
    Lt = Lr + La + t * Lw
    
    return Lt

# Create TOA radiances for different water types
Lt_clear = create_synthetic_toa_radiance('clear_ocean')
Lt_productive = create_synthetic_toa_radiance('productive')
Lt_turbid = create_synthetic_toa_radiance('turbid')

print("Synthetic TOA Radiances (clear ocean):")
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]
for wl, Lt in zip(wavelengths, Lt_clear):
    print(f"  {wl}nm: {Lt:.3f} mW/cm^2/um/sr")

In [ ]:
# Visualize TOA spectra for different water types
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]

plt.figure(figsize=(10, 6))
plt.plot(wavelengths, Lt_clear, 'b-o', linewidth=2, 
         markersize=8, label='Clear Ocean')
plt.plot(wavelengths, Lt_productive, 'g-s', linewidth=2, 
         markersize=8, label='Productive')
plt.plot(wavelengths, Lt_turbid, 'brown', marker='^', linewidth=2, 
         markersize=8, label='Turbid')

plt.xlabel('Wavelength (nm)', fontsize=12)
plt.ylabel('TOA Radiance (mW/cm²/µm/sr)', fontsize=12)
plt.title('Synthetic TOA Radiance Spectra for Different Water Types', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 3.2 Define Viewing Geometry

In [ ]:
# Create geometry angles for a typical satellite observation
geometry = GeometryAngles(
    solar_zenith=35.0,      # Sun 35 degrees from zenith
    solar_azimuth=135.0,    # Sun in SE quadrant
    view_zenith=20.0,       # Sensor 20 degrees from nadir
    view_azimuth=270.0      # Sensor looking West
)

# Calculate relative azimuth
rel_azimuth = abs(geometry.solar_azimuth - geometry.view_azimuth)
if rel_azimuth > 180:
    rel_azimuth = 360 - rel_azimuth

print("Viewing Geometry:")
print(f"  Solar zenith angle: {geometry.solar_zenith}°")
print(f"  Solar azimuth angle: {geometry.solar_azimuth}°")
print(f"  View zenith angle: {geometry.view_zenith}°")
print(f"  View azimuth angle: {geometry.view_azimuth}°")
print(f"  Relative azimuth: {rel_azimuth}°")

### 3.3 Define Ancillary Data

In [ ]:
# Create ancillary data for typical conditions
ancillary = AncillaryData(
    pressure=1015.0,              # Surface pressure (hPa)
    wind_speed=6.0,               # 10m wind speed (m/s)
    ozone=280.0,                  # Total column ozone (Dobson Units)
    water_vapor=2.0,              # Precipitable water vapor (g/cm²)
    relative_humidity=75.0,       # Relative humidity (%)
    no2_total=4.0e15,             # Total NO2 column (molecules/cm²)
    no2_stratospheric=2.5e15      # Stratospheric NO2 (molecules/cm²)
)

print("Ancillary Data:")
print(f"  Surface pressure: {ancillary.pressure} hPa")
print(f"  Wind speed: {ancillary.wind_speed} m/s")
print(f"  Ozone: {ancillary.ozone} DU")
print(f"  Water vapor: {ancillary.water_vapor} g/cm²")
print(f"  Relative humidity: {ancillary.relative_humidity}%")
print(f"  NO2 (total): {ancillary.no2_total:.2e} molec/cm²")
print(f"  NO2 (stratospheric): {ancillary.no2_stratospheric:.2e} molec/cm²")

---
## 4. Run the Atmospheric Correction

The `process()` method performs the full atmospheric correction.

In [ ]:
# Run atmospheric correction for clear ocean case
result = ac.process(Lt_clear, geometry, ancillary)

print("Atmospheric Correction Complete!")
print(f"\nResult type: {type(result).__name__}")

---
## 5. Understanding the Outputs

The `CorrectionResult` object contains several output products.

### 5.1 Remote Sensing Reflectance (Rrs)

Rrs is the primary ocean color product, defined as:

$$R_{rs} = \frac{L_w}{E_d}$$

where $E_d$ is the downwelling irradiance at the surface. Units are sr⁻¹.

In [ ]:
# Display Rrs values
print("Remote Sensing Reflectance (Rrs):")
print("-" * 45)
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]

for i, wl in enumerate(wavelengths):
    rrs_val = result.rrs[i]
    print(f"  Band {i+1} ({wl}nm): Rrs = {rrs_val:.6f} sr⁻¹")

In [ ]:
# Plot Rrs spectrum
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]
rrs_values = result.rrs

plt.figure(figsize=(10, 6))
plt.plot(wavelengths, rrs_values, 'b-o', linewidth=2, markersize=10)
plt.fill_between(wavelengths, 0, rrs_values, alpha=0.2)
plt.xlabel('Wavelength (nm)', fontsize=12)
plt.ylabel('Rrs (sr⁻¹)', fontsize=12)
plt.title('Remote Sensing Reflectance Spectrum', fontsize=14)
plt.grid(True, alpha=0.3)
plt.xlim(400, 900)
plt.tight_layout()
plt.show()

### 5.2 Normalized Water-Leaving Radiance (nLw)

nLw is the water-leaving radiance normalized to reference conditions (sun at zenith, standard Earth-Sun distance).

In [ ]:
# Display nLw values
print("Normalized Water-Leaving Radiance (nLw):")
print("-" * 50)
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]

for i, wl in enumerate(wavelengths):
    nlw_val = result.nLw[i]
    print(f"  Band {i+1} ({wl}nm): nLw = {nlw_val:.4f} mW/cm²/µm/sr")

### 5.3 Aerosol Properties

In [ ]:
# Aerosol path radiance
print("Aerosol Path Radiance (La):")
print("-" * 45)
wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]

for i, wl in enumerate(wavelengths):
    la_val = result.La[i]
    print(f"  Band {i+1} ({wl}nm): La = {la_val:.4f}")

print(f"\nAerosol Optical Thickness at 865nm: {result.taua[-1]:.4f}")
print(f"Angstrom Exponent: {result.angstrom:.3f}")

### 5.4 Chlorophyll Concentration

The OC3/OC4 algorithm estimates chlorophyll-a concentration from band ratios.

In [ ]:
print(f"Estimated Chlorophyll-a: {result.chlorophyll:.3f} mg/m³")

---
## 6. Quality Flags

The correction produces quality flags to indicate potential issues.

In [ ]:
# Check quality flags
flags = result.flags

print("Quality Flags:")
print("-" * 40)
print(f"  Glint masked: {flags.glint_masked}")
print(f"  Atmospheric correction warning: {flags.atmospheric_correction_warning}")
print(f"  Negative water-leaving radiance: {flags.negative_water_leaving}")
print(f"  Iteration limit reached: {flags.iteration_limit_reached}")
print(f"  Turbid water (non-black NIR): {flags.turbid_water}")
print(f"  Straylight contamination: {flags.straylight}")
print(f"  Cloud detected: {flags.cloud}")

---
## 7. Compare Different Water Types

Let's process all three water types and compare the Rrs spectra.

In [ ]:
# Process all water types
result_clear = ac.process(Lt_clear, geometry, ancillary)
result_productive = ac.process(Lt_productive, geometry, ancillary)
result_turbid = ac.process(Lt_turbid, geometry, ancillary)

# Extract Rrs values (now ndarrays)
rrs_clear = result_clear.rrs
rrs_productive = result_productive.rrs
rrs_turbid = result_turbid.rrs

wavelengths = [412, 443, 490, 510, 555, 670, 765, 865]

In [ ]:
# Compare Rrs spectra
plt.figure(figsize=(12, 6))

plt.plot(wavelengths, rrs_clear, 'b-o', linewidth=2, markersize=10, 
         label=f'Clear Ocean (Chl = {result_clear.chlorophyll:.2f} mg/m³)')
plt.plot(wavelengths, rrs_productive, 'g-s', linewidth=2, markersize=10, 
         label=f'Productive (Chl = {result_productive.chlorophyll:.2f} mg/m³)')
plt.plot(wavelengths, rrs_turbid, 'brown', marker='^', linewidth=2, markersize=10, 
         label=f'Turbid (Chl = {result_turbid.chlorophyll:.2f} mg/m³)')

plt.xlabel('Wavelength (nm)', fontsize=12)
plt.ylabel('Rrs (sr⁻¹)', fontsize=12)
plt.title('Retrieved Rrs Spectra for Different Water Types', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.xlim(400, 900)
plt.tight_layout()
plt.show()

### Spectral Shape Interpretation

- **Clear Ocean (blue)**: High blue reflectance, low green/red - typical of oligotrophic waters
- **Productive (green)**: Lower blue, higher green - phytoplankton absorbs blue light
- **Turbid (brown)**: High across visible, extends into NIR - sediment/detritus scattering

---
## 8. Atmospheric Correction Steps

The correction follows this sequence (from NASA TM-2016-217551):

1. **Gas absorption correction** - Remove O3 and NO2 absorption
2. **Polarization correction** - Account for sensor polarization sensitivity
3. **Whitecap removal** - Subtract whitecap contribution
4. **Rayleigh correction** - Remove molecular scattering
5. **Sun glint correction** - Remove/mask specular reflection
6. **Aerosol correction** - Estimate and remove aerosol path radiance
7. **Transmittance correction** - Account for atmospheric attenuation
8. **BRDF correction** - Normalize to standard geometry
9. **Calculate products** - Compute Rrs, nLw, chlorophyll

In [ ]:
# Visualize the correction steps (conceptual)
steps = [
    'TOA Radiance\n(Lt)',
    'Gas\nCorrection',
    'Polarization\nCorrection',
    'Whitecap\nRemoval',
    'Rayleigh\nCorrection',
    'Glint\nCorrection',
    'Aerosol\nCorrection',
    'Transmittance\nCorrection',
    'BRDF\nCorrection',
    'Rrs, nLw\nChlorophyll'
]

fig, ax = plt.subplots(figsize=(14, 3))

# Draw boxes
for i, step in enumerate(steps):
    color = 'lightblue' if i == 0 else ('lightgreen' if i == len(steps)-1 else 'lightyellow')
    rect = plt.Rectangle((i * 1.4, 0), 1.2, 1, facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(i * 1.4 + 0.6, 0.5, step, ha='center', va='center', fontsize=9, fontweight='bold')
    
    # Draw arrows
    if i < len(steps) - 1:
        ax.annotate('', xy=(i * 1.4 + 1.35, 0.5), xytext=(i * 1.4 + 1.25, 0.5),
                    arrowprops=dict(arrowstyle='->', color='black', lw=2))

ax.set_xlim(-0.2, 14)
ax.set_ylim(-0.3, 1.3)
ax.axis('off')
ax.set_title('Atmospheric Correction Processing Chain', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

---
## 9. Working with Arrays (Imagery)

The processor can handle 2D arrays for processing entire images.

In [ ]:
# Create a small synthetic image (10x10 pixels)
ny, nx = 10, 10
n_bands = 8

# Create TOA radiance array with some spatial variation
base_Lt = np.array([9.5, 7.5, 5.2, 4.0, 2.5, 1.0, 0.5, 0.3])

# Add some spatial variation: shape (n_bands, ny, nx)
np.random.seed(42)
Lt_image = base_Lt[:, np.newaxis, np.newaxis] * (1 + 0.1 * np.random.randn(n_bands, ny, nx))

# Create geometry arrays
geometry_image = GeometryAngles(
    solar_zenith=np.full((ny, nx), 35.0),
    solar_azimuth=np.full((ny, nx), 135.0),
    view_zenith=np.full((ny, nx), 20.0),
    view_azimuth=np.full((ny, nx), 270.0)
)

# Create ancillary arrays
ancillary_image = AncillaryData(
    pressure=np.full((ny, nx), 1015.0),
    wind_speed=np.full((ny, nx), 6.0),
    ozone=np.full((ny, nx), 280.0),
    water_vapor=np.full((ny, nx), 2.0),
    relative_humidity=np.full((ny, nx), 75.0),
    no2_total=np.full((ny, nx), 4.0e15),
    no2_stratospheric=np.full((ny, nx), 2.5e15)
)

print(f"Created synthetic image: {ny} x {nx} pixels")
print(f"Lt_image shape: {Lt_image.shape} (n_bands, ny, nx)")

In [ ]:
# Process the image
result_image = ac.process(Lt_image, geometry_image, ancillary_image)

print("Image processing complete!")
print(f"Output Rrs shape: {result_image.rrs.shape}")

In [ ]:
# Visualize some output bands
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

band_indices = [0, 2, 4]  # Bands 1, 3, 5
wavelengths_to_show = [412, 490, 555]

for i, (band_idx, wl) in enumerate(zip(band_indices, wavelengths_to_show)):
    # TOA radiance
    im1 = axes[0, i].imshow(Lt_image[band_idx], cmap='viridis')
    axes[0, i].set_title(f'TOA Lt ({wl}nm)')
    plt.colorbar(im1, ax=axes[0, i], shrink=0.8)
    
    # Retrieved Rrs
    im2 = axes[1, i].imshow(result_image.rrs[band_idx], cmap='viridis')
    axes[1, i].set_title(f'Rrs ({wl}nm)')
    plt.colorbar(im2, ax=axes[1, i], shrink=0.8)

plt.suptitle('Input TOA Radiance vs Retrieved Rrs', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Show chlorophyll map
plt.figure(figsize=(8, 6))
im = plt.imshow(result_image.chlorophyll, cmap='YlGn')
plt.colorbar(im, label='Chlorophyll-a (mg/m³)')
plt.title('Retrieved Chlorophyll-a Concentration', fontsize=14)
plt.tight_layout()
plt.show()

---
## Summary

This notebook demonstrated the complete atmospheric correction workflow:

1. **Setup**: Initialize `AtmosphericCorrection` for your sensor
2. **Inputs**: Prepare TOA radiances, geometry, and ancillary data
3. **Processing**: Call `process()` to run the full correction
4. **Outputs**: Retrieve Rrs, nLw, chlorophyll, aerosol properties, and flags

### Key Products

| Product | Description | Units |
|---------|-------------|-------|
| Rrs | Remote sensing reflectance | sr⁻¹ |
| nLw | Normalized water-leaving radiance | mW/cm²/µm/sr |
| La | Aerosol path radiance | mW/cm²/µm/sr |
| taua | Aerosol optical thickness | dimensionless |
| chlorophyll | Chlorophyll-a concentration | mg/m³ |

### References

- Mobley, C.D., et al. (2016). Ocean Optics Web Book. NASA Technical Memorandum TM-2016-217551.
- Gordon, H.R. and Wang, M. (1994). Retrieval of water-leaving radiance and aerosol optical thickness over the oceans with SeaWiFS.
- Ahmad, Z., et al. (2010). New aerosol models for the retrieval of aerosol optical thickness and normalized water-leaving radiances from the SeaWiFS and MODIS sensors.